# Keypoint Detection of a Tennis Court

Based on : https://github.com/yastrebksv/TennisCourtDetector?tab=readme-ov-file

## Dataset

The Dataset was taken from the GitHub repository and consists of 8841 images, which were separeted to train set (75%) and validation set (25%). Each image has 14 annotated points. The resolution of images is 1280×720. This dataset contains all court types (hard, clay and grass).



In [1]:
!wget --header="Host: drive.usercontent.google.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7" --header="Accept-Language: en-US,en;q=0.9,ar;q=0.8" --header="Cookie: HSID=Ag2OIHvsd2Wub4C7z; SSID=AWnBcQKwDHiTrZAU1; APISID=pltrFZgE9lJ0o1gq/AN9feEHYvs8oHd519; SAPISID=zgF45F21ZPWzYWZw/AgUMJ8b7QQXuWGn19; __Secure-1PAPISID=zgF45F21ZPWzYWZw/AgUMJ8b7QQXuWGn19; __Secure-3PAPISID=zgF45F21ZPWzYWZw/AgUMJ8b7QQXuWGn19; SID=g.a000fwgYx1PcnW-rFyFhg3x6mQHzCrwXz-KFhoOLogUl7YTWI-uttBbVDRolhF-hY16nwHXw0gACgYKAWISAQASFQHGX2MivNTw_E_toJuIRy6LMpKNOBoVAUF8yKpFSmvq7AMjvEWeNc50Zff40076; __Secure-1PSID=g.a000fwgYx1PcnW-rFyFhg3x6mQHzCrwXz-KFhoOLogUl7YTWI-utbSY2jBY1VXuw8gYl5hIO2QACgYKAXsSAQASFQHGX2MihVCJ1PwLozGqZgdSatM9QhoVAUF8yKpgrsTvI8i_UE-YHpoN7Gx-0076; __Secure-3PSID=g.a000fwgYx1PcnW-rFyFhg3x6mQHzCrwXz-KFhoOLogUl7YTWI-utwVfPl2imdPimZJ9tdDZGQAACgYKAUESAQASFQHGX2MiEJ49mV4jME2kttDAV5hwWBoVAUF8yKp80mIgju1lu-q4nI7VsFDM0076; NID=511=efI9IZpxtyJ7Dw1MAUXU8FlzS5jXGewY4Er8HliWc3A0RSWdgvNDyKY66ETjgRyTGWPbWODSmiSeYSBab5SPHVwqbJxd6ZeGW2f6BkHi61UKksXPH0CVJRM1hKpMjHPU5qw7tboM2Mi87NrosV8COB-GCLulLLbjOoSAEQewTe8NVZ5Owq8IkwvxFGfJkmUKEMkFWrw9yb5nTDl3wbZEsGFI92iEdNTSxSRovNCIPN2US-SCFdQ0m2BtvwdiWZbgnn7dSQ8yPA145Kk2BA-ATpJNJ6SJHEHLQY-9CPail9D5qgJgxR925EUg5RGCpEu9wS5xbA62KTa19wAvbAq7Dk3TWc-iX4p1s7ESFyDC7yMpFxiFPJjqkWwFi_ZfiK2TW2t0TQ60DFBxqOytQaLyHrkEvD-CQPVj6OCOP22cZY0Cu61HaAQgFO9pXH-kJUlywzVdbirJumN5gswyaQ49b3KdLcG0Jb7brOMTM24T2nGtQ10hJzsnTwX7dBk3ujqQrI_DGuURvPassPUrIZ0; AEC=Ae3NU9MOEGeKAZjP6INpOYbyMraWAWztmx5pJB_1ILu1furiTy1K37k15u0; __Secure-1PSIDTS=sidts-CjEBYfD7Z9twEKTWJ9gU7KG-rLbxJGNRQIoG3wH6JVu6yiCC2fsRrm7tN8L6d5WlILrnEAA; __Secure-3PSIDTS=sidts-CjEBYfD7Z9twEKTWJ9gU7KG-rLbxJGNRQIoG3wH6JVu6yiCC2fsRrm7tN8L6d5WlILrnEAA; 1P_JAR=2024-02-18-08; SIDCC=ABTWhQExCxkfmwCkG1RaEgz8U1ZkPeh3HmLMUdMt8S5cNSsLY5U5rAL6wlvq7dtjRw7zrtAbqsFI; __Secure-1PSIDCC=ABTWhQH0jLeRIS6Tu3LS8DXB5Q3gGDq9LTmlk60FKu795Bf0UbzsOcYWVAE96clq5aAL8i724Q0; __Secure-3PSIDCC=ABTWhQHIFcyv3nZYwp78WXEQal71jCE_ZsGT5lXs8VLr7XDIfFqHcLTIPz4HxzJb9ZnYQ5l2s9eU" --header="Connection: keep-alive" "https://drive.usercontent.google.com/download?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&export=download&authuser=0&confirm=t&uuid=3077628e-fc9b-4ef2-8cde-b291040afb30&at=APZUnTU9lSikCSe3NqbxV5MVad5T%3A1708243355040" -c -O 'tennis_court_det_dataset.zip'


--2025-02-22 14:58:03--  https://drive.usercontent.google.com/download?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&export=download&authuser=0&confirm=t&uuid=3077628e-fc9b-4ef2-8cde-b291040afb30&at=APZUnTU9lSikCSe3NqbxV5MVad5T%3A1708243355040
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.101.132, 2607:f8b0:4023:c06::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.101.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7255696316 (6.8G) [application/octet-stream]
Saving to: ‘tennis_court_det_dataset.zip’

tennis_court_det_da 100%[===================>]   6.76G  87.4MB/s    in 97s     

2025-02-22 14:59:43 (71.5 MB/s) - ‘tennis_court_det_dataset.zip’ saved [7255696316/7255696316]



In [2]:
!unzip tennis_court_det_dataset.zip

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
  inflating: data/images/JNKp7sCdQlY_2200.png  
  inflating: data/images/JNKp7sCdQlY_2250.png  
  inflating: data/images/JNKp7sCdQlY_250.png  
  inflating: data/images/JNKp7sCdQlY_300.png  
  inflating: data/images/JNKp7sCdQlY_350.png  
  inflating: data/images/JNKp7sCdQlY_400.png  
  inflating: data/images/JNKp7sCdQlY_450.png  
  inflating: data/images/JNKp7sCdQlY_50.png  
  inflating: data/images/JNKp7sCdQlY_500.png  
  inflating: data/images/JNKp7sCdQlY_550.png  
  inflating: data/images/JNKp7sCdQlY_600.png  
  inflating: data/images/JNKp7sCdQlY_650.png  
  inflating: data/images/JNKp7sCdQlY_700.png  
  inflating: data/images/JNKp7sCdQlY_750.png  
  inflating: data/images/juXbdW7z0WA_100.png  
  inflating: data/images/juXbdW7z0WA_1050.png  
  inflating: data/images/juXbdW7z0WA_1100.png  
  inflating: data/images/juXbdW7z0WA_200.png  
  inflating: data/images/juXbdW7z0WA_350.png  
  inflating: data/images/juXbdW7z0WA_

## Model Training

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
from torchvision import models, transforms
import json
import numpy as np

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
class KeyPointsDataset(Dataset):
    def __init__(self, img_dir,data_file):
        self.img_dir = img_dir
        with open(data_file, 'r') as f:
            self.data = json.load(f)

        self.transforms = transforms.Compose(
            [transforms.ToPILImage(),
            # Resize for faster training (TODO: ?)
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            # Normalize using the mean & std of ImageNet (since we use a pretrained
            # ResNet50 model, which was trained on ImageNet)
            transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])]
        )


    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        img_h, img_w = img.shape[0], img.shape[1]

        # cv2 reads in BGR, so we convert
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        # Get Keypoints as 1D array of floats
        img_kps = np.array(item['kps']).flatten()
        img_kps = img_kps.astype(np.float32)

        # Convert keypoint positions to new scale (since we rescaled the image)

        # x coordinates (stored at 0,2,4,...)
        img_kps[::2] *= 224.0 /img_w
        # y coordinates (stored at 1,3,5,...)
        img_kps[1::2] *= 224.0 /img_h

        return img, img_kps







In [24]:
train_dataset = KeyPointsDataset(img_dir = '/content/data/images', data_file = '/content/data/data_train.json')
val_dataset = KeyPointsDataset(img_dir = '/content/data/images', data_file = '/content/data/data_val.json' )

train_loader = DataLoader(train_dataset, batch_size = 8, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 8, shuffle = True)

For the model, we will finetune a pretrained ResNet50 model.

In [18]:
model = models.resnet50(pretrained = True)

# Fine-tuning only the last fully connected layer to output 14 keypoints (each has
# a x & y coordinate, therefore in total 14*2 values)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2)


model = model.to(device = device)



/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 144MB/s]


In [19]:
criterion = torch.nn.MSELoss() # as we have a regression task (estimating coordinates), we can use MSELoss
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

In [26]:
epochs = 20

for epoch in range(epochs):
    for i, (imgs, imgs_kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        imgs_kps = imgs_kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, imgs_kps) # MSE between predicted & actual keypoints
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch : {epoch} , Step : {i} , Loss : {loss.item()}")

Epoch : 0 , Step : 0 , Loss : 13588.2861328125
Epoch : 0 , Step : 10 , Loss : 13299.11328125
Epoch : 0 , Step : 20 , Loss : 12926.2236328125
Epoch : 0 , Step : 30 , Loss : 12602.6533203125
Epoch : 0 , Step : 40 , Loss : 12303.4111328125
Epoch : 0 , Step : 50 , Loss : 11913.7099609375
Epoch : 0 , Step : 60 , Loss : 11441.005859375
Epoch : 0 , Step : 70 , Loss : 11175.4404296875
Epoch : 0 , Step : 80 , Loss : 10762.8017578125
Epoch : 0 , Step : 90 , Loss : 10234.251953125
Epoch : 0 , Step : 100 , Loss : 10304.951171875
Epoch : 0 , Step : 110 , Loss : 10069.9091796875
Epoch : 0 , Step : 120 , Loss : 9859.162109375
Epoch : 0 , Step : 130 , Loss : 8999.9169921875
Epoch : 0 , Step : 140 , Loss : 9523.8408203125
Epoch : 0 , Step : 150 , Loss : 9201.4521484375
Epoch : 0 , Step : 160 , Loss : 8090.56689453125
Epoch : 0 , Step : 170 , Loss : 7575.1572265625
Epoch : 0 , Step : 180 , Loss : 7885.13134765625
Epoch : 0 , Step : 190 , Loss : 7374.48046875
Epoch : 0 , Step : 200 , Loss : 7403.20898437

In [30]:
# Test score on validation set (i.e. we just use as a test set here )

model.eval()
total_loss = 0

for i, (imgs, imgs_kps) in enumerate(val_loader):
    imgs = imgs.to(device)
    imgs_kps = imgs_kps.to(device)

    outputs = model(imgs)
    loss = criterion(outputs, imgs_kps)

    total_loss += loss.item()

print(f"Validation Loss : {total_loss / len(val_loader)}")


Validation Loss : 6.351386767754916


In [28]:
torch.save(model.state_dict(), "keypoints_model.pth")